In [47]:
from googleapiclient.discovery import build

In [49]:
youtube=build('youtube', 'v3', developerKey=apiKey)

In [ ]:
request= youtube.channels().list(part='statistics', forUsername='CNN')
response=request.execute()
print(response)


In [61]:
exitdata=response['items'][0]['statistics']
exitdata['channel']='CNN'
print(exitdata)


{'viewCount': '18038639275', 'subscriberCount': '18100000', 'hiddenSubscriberCount': False, 'videoCount': '171926', 'channel': 'CNN'}


In [51]:
import pandas as pd

In [63]:
data=pd.DataFrame([exitdata])
print(data)

     viewCount subscriberCount  hiddenSubscriberCount videoCount channel
0  18038639275        18100000                  False     171926     CNN


In [ ]:
from googleapiclient.discovery import build
import json

# Your API key
API_KEY = ''

# Build the YouTube API client
youtube = build('youtube', 'v3', developerKey=API_KEY)
rwvideodt=[]
# Function to get all videos from a channel using channel ID
def get_all_videos(channel_id):
    try:
        # Get the "Uploads" playlist ID for the given channel ID
        playlists_request = youtube.channels().list(part="contentDetails", id=channel_id)
        playlists_response = playlists_request.execute()
        
        # Check if we got the response successfully
        if playlists_response['pageInfo']['totalResults'] > 0:
            # The "uploads" playlist ID for the channel
            upload_playlist_id = playlists_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
            
            # Get all videos in the uploads playlist
            videos = []
            next_page_token = None
            

            while True:
                # Get videos from the playlist
                playlist_items_request = youtube.playlistItems().list(
                    part="snippet", 
                    playlistId=upload_playlist_id,
                    maxResults=50,  # You can adjust this value
                    pageToken=next_page_token
                )
                playlist_items_response = playlist_items_request.execute()
                rwvideodt.append(playlist_items_response)

                # Collect video details
                for item in playlist_items_response['items']:
                    video_id = item['snippet']['resourceId']['videoId']
                    video_title = item['snippet']['title']
                    video_description = item['snippet']['description']
                    video_publish_date = item['snippet']['publishedAt']
                    
                    # Append video details to list
                    videos.append({
                        'video_id': video_id,
                        'title': video_title,
                        'description': video_description,
                        'publish_date': video_publish_date
                    })
                
                # Check if there is another page of results
                next_page_token = playlist_items_response.get('nextPageToken')
                if not next_page_token:
                    break
            
            return videos
        else:
            print("No uploads found for this channel.")
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Example Usage: Get all videos from a channel using Channel ID
channel_id = 'UCqUyE0GB1heTwJL7jKRhffQ'  # Replace with the actual channel ID
videos = get_all_videos(channel_id)

if videos:
    for video in videos:
        print(json.dumps(video,indent=4))
        
else:
    print("No videos found.")


{
    "video_id": "WcS3TLjVok4",
    "title": "\u3010FULL\u3011His System Makes Him OP! He Helps A Princess Disguised The Throne\u2014And Her Heart!#minidrama",
    "description": "\ud83d\udd25Rush MiniDrama | MiniDrama for Men Who Love to Win\u200b\ud83d\udd25\nWelcome to Rush MiniDrama! \ud83d\udca5\n\ud83e\udd20Get ready for high-speed thrills, intense action, and jaw-dropping twists\u2014all packed into bite-sized dramas made just for you! From revenge and power struggles to unexpected betrayals, Rush MiniDrama brings you the most exciting short stories that keep your heart racing\uff01\ud83e\udef5\n\u27a1\ufe0fOfficially Licensed \u2013 Unauthorized use will be pursued! \nDon\u2019t miss out\u2014subscribe now and experience the rush! \ud83d\udc40\n\n\u2764\ufe0fThanks to all the subscribers\uff1ahttps://bit.ly/4i0SSzg\n\n#minidrama#fyp#chinesedrama#mandrama#godwar#revenge#miracle",
    "publish_date": "2025-05-22T06:41:28Z"
}
{
    "video_id": "JbW0l_iLgm0",
    "title": "\u3010F

In [3]:
import json
with open('youtubejsonfile.json','w') as file:
    json.dump(videos,file,indent=4)


In [ ]:
print(rwvideodt)
for video in rwvideodt:
    print(json.dumps(video,indent=4))

In [ ]:
import mysql.connector
from mysql.connector import Error

# Replace with your MySQL credentials
host = "localhost"       # MySQL host (usually "localhost")
user = ""          # MySQL username
password = ""  # MySQL password
database = "youtube"  # The database you want to connect to

try:
    # Establish the connection
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Check if the connection was successful
    if connection.is_connected():
        print("Successfully connected to MySQL database")

        # Create a cursor to interact with the database
        cursor = connection.cursor()

        for video in videos:

            # Example: Insert data into a table
            sql_insert_query = """INSERT INTO video (video_id ,title,description, publish_date )
                                VALUES (%s, %s, %s, %s)"""
            
            # Data to insert
            data_to_insert = (video['video_id'],video['title'],video['description'],video['publish_date'],)

            # Execute the INSERT statement
            cursor.execute(sql_insert_query, data_to_insert)

        # Commit the transaction
        connection.commit()
        print("Data inserted successfully")

except Error as e:
    print(f"Error while connecting to MySQL: {e}")
    connection.rollback()  # Rollback in case of error

finally:
    # Close the cursor and connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")


Successfully connected to MySQL database
Data inserted successfully
MySQL connection closed.
